# Notebook for CHEM 4582 Biochemistry II Lab Biometals CURE
Goal: Analyzing variants in heme transport genes in different populations from the 1000 Genomes Project.


README: 

# Organization

In [1]:
#make a directory for the vcf files
!mkdir ~/biometals/VCFfiles

In [1]:
import pandas as pd
import gzip

In [1]:
#--------------------------------------------------------------
#creating text files for the european populations
#--------------------------------------------------------------
filename = 'europops.txt'

# Open the file in write mode ('w') and write content
with open(filename, 'w') as file:
    file.write("FIN.\n") #Finnish in Finland
    file.write("GBR.\n") #British in Great Britain and Scotland
    file.write("IBR.\n") #Iberian in Spain
    file.write("TSI.\n") #Toscani in Italy
print(f"{filename} has been created and written to biometals.")

europops.txt has been created and written to.


In [1]:
!mv ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz ~/biometals/VCFfiles

In [1]:
!bcftools


Program: bcftools (Tools for variant calling and manipulating VCFs and BCFs)
License: GNU GPLv3+, due to use of the GNU Scientific Library
Version: 1.17 (using htslib 1.17)

Usage:   bcftools [--version|--version-only] [--help] <command> <argument>

Commands:

 -- Indexing
    index        index VCF/BCF files

 -- VCF/BCF manipulation
    annotate     annotate and edit VCF/BCF files
    concat       concatenate VCF/BCF files from the same set of samples
    convert      convert VCF/BCF files to different formats and back
    head         view VCF/BCF file headers
    isec         intersections of VCF/BCF files
    merge        merge VCF/BCF files files from non-overlapping sample sets
    norm         left-align and normalize indels
    plugin       user-defined plugins
    query        transform VCF/BCF into user-defined formats
    reheader     modify VCF/BCF header, change sample names
    sort         sort VCF/BCF file
    view         VCF/BCF conversion, view, subset and filter V

# HBB Variant (Sickle Cell Anemia)

In [1]:
#--------------------------------------------------------------
# calling the chromosome 11 data
#--------------------------------------------------------------
!wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

--2024-11-12 20:18:42--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 734583405 (701M) [application/x-gzip]
Saving to: ‘ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’

ALL.chr11.phase3_sh 100%[===================>] 700.55M  70.5MB/s    in 9.4s    

2024-11-12 20:18:52 (74.6 MB/s) - ‘ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz’ saved [734583405/734583405]



In [1]:
#index the file for ease of access
!bcftools index ~/biometals/VCFfiles/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

In [1]:
# checking if the variant is present
!bcftools view -r 11:5291019-5291019 ~/biometals/VCFfiles/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz

##fileformat=VCFv4.1
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=20150218
##reference=ftp://ftp.1000genomes.ebi.ac.uk//vol1/ftp/technical/reference/phase2_reference_assembly_sequence/hs37d5.fa.gz
##source=1000GenomesPhase3Pipeline
##contig=<ID=1,assembly=b37,length=249250621>
##contig=<ID=2,assembly=b37,length=243199373>
##contig=<ID=3,assembly=b37,length=198022430>
##contig=<ID=4,assembly=b37,length=191154276>
##contig=<ID=5,assembly=b37,length=180915260>
##contig=<ID=6,assembly=b37,length=171115067>
##contig=<ID=7,assembly=b37,length=159138663>
##contig=<ID=8,assembly=b37,length=146364022>
##contig=<ID=9,assembly=b37,length=141213431>
##contig=<ID=10,assembly=b37,length=135534747>
##contig=<ID=11,assembly=b37,length=135006516>
##contig=<ID=12,assembly=b37,length=133851895>
##contig=<ID=13,assembly=b37,length=115169878>
##contig=<ID=14,assembly=b37,length=107349540>
##contig=<ID=15,assembly=b37,length=102531392>
##contig=<ID=16,assembly=b37,length=90354753>
##contig

In [ ]:
#--------------------------------------------------------------
# rs334 variant information
#--------------------------------------------------------------
# located on chr11
# position 5291019 (GRCh38)
# ref allele: A
# alt allele: T

In [1]:
#--------------------------------------------------------------
# creating text files containing the sample IDs in each population
#--------------------------------------------------------------
import pandas as pd
# load and filter the population file
panel_file = '~/biometals/VCFfiles/integrated_call_samples_v3.20130502.ALL.panel' 
panel_df = pd.read_csv(panel_file, sep='\t') 

# Filter for the target populations
target_populations = ['FIN', 'GBR', 'IBS', 'TSI']
filtered_samples = panel_df[panel_df['pop'].isin(target_populations)]

# Save sample lists for each population
sample_dict = {}
for pop in target_populations:
    sample_ids = filtered_samples[filtered_samples['pop'] == pop]['sample'].tolist()
    # Save to a text file for BCFtools
    sample_file = f"{pop}_samples.txt"
    with open(sample_file, 'w') as file:
        for sample in sample_ids:
            file.write(sample + '\n')
    sample_dict[pop] = sample_file

{'FIN': 0, 'GBR': 0, 'IBS': 0, 'TSI': 0}

In [ ]:
#-----------------------------------------------------

In [1]:
#--------------------------------------------------------------
# load in the tbi file for chr11
#--------------------------------------------------------------
!wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/
        ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi

--2024-11-13 14:24:43--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133030 (130K) [application/x-gzip]
Saving to: ‘ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi’

ALL.chr11.phase3_sh 100%[===================>] 129.91K   460KB/s    in 0.3s    

2024-11-13 14:24:44 (460 KB/s) - ‘ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi’ saved [133030/133030]



In [1]:
!mv ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi ~/biometals/VCFfiles

In [1]:
!cat ~/biometals/VCFfiles/ALL.chr11.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz.tbi | grep -v "##" | cut -f1-5,11-16 | head

grep: (standard input): binary file matches


In [1]:
#--------------------------------------------------------------
#extract identifiers into a text file
#--------------------------------------------------------------
!bcftools query -l ~/biometals/VCFfiles/rs334_subset.vcf > ~/biometals/VCFfiles/rs334_identifiers.txt

In [1]:
#--------------------------------------------------------------
# extracting sample and population data from the panel file into a CSV file, samplepopdata.csv
# contains all samples (individuals)
#--------------------------------------------------------------
import pandas as pd
popinfo = pd.read_csv('~/biometals/VCFfiles/integrated_call_samples_v3.20130502.ALL.panel', sep='\t')
print(popinfo.columns)
samplepopdata = popinfo[['sample', 'pop']]
#create a 2-column csv with sample and pop
samplepopdata.to_csv('samplepopdata.csv', index=False)
print("done")

#cross reference the identifiers to the sample populations
#load population
popmap = pd.read_csv('samplepopdata.csv')
# Load identifiers
rs334identifiers = pd.read_csv('~/biometals/VCFfiles/rs334_identifiers.txt', header=None, names=['identifier'])
print("done")

#merge and save as a csv
merged_rs334_data = pd.merge(rs334identifiers, popmap, left_on='identifier', right_on='sample', how='left')
merged_rs334_data.to_csv('merged_rs334_data.csv', index=False)
print("done")

#count the number in each population
rs334_population_counts = merged_rs334_data.groupby('pop').size().reset_index(name='counts')
print(rs334_population_counts)
print("done")

Index(['sample', 'pop', 'super_pop', 'gender', 'Unnamed: 4', 'Unnamed: 5'], dtype='object')
done
done
done
    pop  counts
0   ACB      96
1   ASW      61
2   BEB      86
3   CDX      93
4   CEU      99
5   CHB     103
6   CHS     105
7   CLM      94
8   ESN      99
9   FIN      99
10  GBR      91
11  GIH     103
12  GWD     113
13  IBS     107
14  ITU     102
15  JPT     104
16  KHV      99
17  LWK      99
18  MSL      85
19  MXL      64
20  PEL      85
21  PJL      96
22  PUR     104
23  STU     102
24  TSI     107
25  YRI     108
done


In [1]:
# make new directories to order stuff
!mkdir ~/biometals/VCFfiles/identifiers
!mkdir ~/biometals/VCFfiles/subsets

In [1]:
!cat ~/biometals/rs334_output.vcf | grep -v "##" | cut -f1-5,11-16 | head

#CHROM	POS	ID	REF	ALT	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103


---------------------------------------------------------------------------------------------------------------------------









Basically what's happening here is that the file directly from the 1000 Genomes website, for whatever reason, isn't showing the individual allele data for me. 
So, instead of calling it from the website, I use a different method below of using the pre-loaded file in my Genomics and Applied Bioinformatics class, which contains the Ch38 phase 3 data (the same data) in order to complete this study. 
Theoretically, the files are the same on the website (http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/) and in our shared folder, so it shouldn't make a difference if someone doesn't have access to this folder. At least, as far as I understand this is the case. 








In [ ]:
#------------------------------
# 1) so that file was funky so we're gonna try the GAAB file instead
#------------------------------

the following lines of code were pulled from a DemoNotebook in BIOS4150/BIOL6150

In [1]:
#Set up working directory.
!mkdir -p ~/biometals/AnnotateVariants

In [ ]:
#Extract HBB gene area for Sickle Cell Disease
#the VCF files are loaded into our notebooks through my BIOL 4150 class so I just call it from there
!bcftools view -r chr11:5225464-5227071 -Ov -o ~/biometals/AnnotateVariants/1000Genomes.Chr11.HBB.vcf 
    ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz

In [1]:
#Check the file.
!cat ~/biometals/AnnotateVariants/1000Genomes.Chr11.HBB.vcf | grep -v "##" | cut -f1-5,11-16 | head

#CHROM	POS	ID	REF	ALT	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103
chr11	5225469	rs528009939	A	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225488	rs33978907	A	G	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225502	rs34029390	A	G	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225542	rs537944366	T	C	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225564	rs200399660	C	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225610	rs36020563	G	A	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225640	rs113082294	C	G	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225653	rs111645889	G	A	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5225660	rs33971634	G	A	0|0	0|0	0|0	0|0	0|0	0|0


The following code was NOT pulled from the DemoNotebook. ChatGPT was used as an aid for creating the following code. 

In [1]:
#--------------------------------------------------------------
# 2) creating text files containing the sample IDs in each population
# this can be done using the website population file, since it contains the same data set information
# this isn't actually used
#--------------------------------------------------------------
import pandas as pd
# load and filter the population file
panel_file = '~/biometals/VCFfiles/integrated_call_samples_v3.20130502.ALL.panel' 
panel_df = pd.read_csv(panel_file, sep='\t') 

# Filter for the target populations
target_populations = ['FIN', 'GBR', 'IBS', 'TSI']
filtered_samples = panel_df[panel_df['pop'].isin(target_populations)]

# Save sample lists for each population
sample_dict = {}
for pop in target_populations:
    sample_ids = filtered_samples[filtered_samples['pop'] == pop]['sample'].tolist()
    # Save to a text file for BCFtools
    sample_file = f"{pop}_samples.txt"
    with open(sample_file, 'w') as file:
        for sample in sample_ids:
            file.write(sample + '\n')
    sample_dict[pop] = sample_file

In [1]:
#rename the file
!mv ~/biometals/AnnotateVariants/1000Genomes.Chr11.HBB.vcf ~/biometals/AnnotateVariants/chr11HBB.vcf

In [ ]:
#--------------------------------------------------------------
# next step: query the file
# this is done on the original filtered VCF for HBB data
#--------------------------------------------------------------

In [ ]:
# query on the non-filtered data
!bcftools query -f'[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs334" && GT="alt"' 
    ~/biometals/AnnotateVariants/chr11HBB.vcf > ~/biometals/AnnotateVariants/full_SickleCellCarrierInfo.txt

In [ ]:
#--------------------------------------------------------------
# next step: look at the whole population data set to see what populations have the variant
#--------------------------------------------------------------

In [1]:
import pandas as pd

# 1) Load the genotype text file containing chromosome position, sample ID, genotype
genotype_file = '~/biometals/AnnotateVariants/full_SickleCellCarrierInfo.txt' 
genotype_df = pd.read_csv(genotype_file, sep='\t', header=None)

# 2) Split the genotype file into separate columns
genotype_df[['position', 'sample', 'genotype']] = genotype_df[0].str.split(' ', expand=True)
# Drop the old column (0) after splitting
genotype_df = genotype_df.drop(columns=[0])

# 3) load samplepopdata
population_file = '~/biometals/samplepopdata.csv'
population_df = pd.read_csv(population_file, sep=',', header=None, names=['sample', 'pop'])

# 4) Merge the genotype data with samplepopdata
merged_df = pd.merge(genotype_df, population_df, on='sample', how='inner')

# 5) Filter for samples with the variation allele
variation_samples = merged_df[merged_df['genotype'].str.contains('1\|1|1\|0|0\|1')]

# 6) Analyze the population distribution of these variation samples
population_counts = variation_samples.groupby('pop').size().reset_index(name='count')
print(population_counts)

   pop  count
0  ACB      9
1  ASW      2
2  CLM      2
3  ESN     24
4  GWD     26
5  LWK     20
6  MSL     21
7  PUR      3
8  YRI     30


In [1]:
#--------------------------------------------------------------
# counting the number of samples per population
#--------------------------------------------------------------
import pandas as pd

# Load the population information file
population_file = '~/biometals/samplepopdata.csv'
population_df = pd.read_csv(population_file, sep=',')

# Count the number of samples in each population
population_counts = population_df.groupby('pop').size().reset_index(name='counts')

# Display the counts for each population
print(population_counts)

    pop  counts
0   ACB      96
1   ASW      61
2   BEB      86
3   CDX      93
4   CEU      99
5   CHB     103
6   CHS     105
7   CLM      94
8   ESN      99
9   FIN      99
10  GBR      91
11  GIH     103
12  GWD     113
13  IBS     107
14  ITU     102
15  JPT     104
16  KHV      99
17  LWK      99
18  MSL      85
19  MXL      64
20  PEL      85
21  PJL      96
22  PUR     104
23  STU     102
24  TSI     107
25  YRI     108


# Beta Thalassemia

Beta thalassemia comes from the HBB gene, which is the same as Sickle Cell Disease and thus the files for
HBB information from above (chr11HBB.vcf) can be used for this study, as can the population information data (samplepopdata.csv)

Positions extracted:
chr11HBBbt.vcf: 5280000-5295500
chr11HBBbt_2.vcf: 5220000-5230000
chr11HBBbt_3.vcf: 113500000-113507000, not used
chr11HBBbt_4.vcf: 5285300-5285400, doesn't contain any allelic data
chr11HBBbt_5.vcf: 5285000-5285500
chr11HBBbt_6.vcf: 

In [1]:
#--------------------------------------------------------------
#Extract HBB gene area for Beta Thalassemia
#--------------------------------------------------------------
#the VCF files are loaded into our notebooks through my BIOS4150/BIOL6150 class so I just call it from there since it's easy
!bcftools view -r chr11:5280000-5295500 -Ov -o ~/biometals/AnnotateVariants/chr11HBBbt.vcf ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz
# this does not contain the variant I want, so two lines down has a different extraction that was actually used

In [1]:
!cat ~/biometals/AnnotateVariants/chr11HBBbt.vcf | grep -v "##" | cut -f1-5,11-16 | head

#CHROM	POS	ID	REF	ALT	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103
chr11	5280105	rs575301686	A	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280222	rs61388411	C	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280270	rs557812923	C	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280277	rs541497770	G	C	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280320	rs60240093	T	A	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280329	rs370648906	C	A	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280344	rs530233431	C	A	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280372	rs542112597	G	T	0|0	0|0	0|0	0|0	0|0	0|0
chr11	5280384	rs563681069	C	A	0|0	0|0	0|0	0|0	0|0	0|0
cut: write error: Broken pipe


#--------------------------------------------------------------
#--------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------
# variant 1) ID==rs33971634 at POS==
#--------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------
# Extracting variant positions for ID==rs33971634
#--------------------------------------------------------------
!bcftools view -r chr11:5220000-5230000 -Ov -o ~/biometals/AnnotateVariants/chr11HBBbt_2.vcf 
    ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz

In [1]:
#--------------------------------------------------------------
# query variant sequence
#--------------------------------------------------------------
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs33971634" && GT="alt"'
    ~/biometals/AnnotateVariants/chr11HBBbt_2.vcf > ~/biometals/AnnotateVariants/var1_BetaThalassemiaInfo.txt

In [1]:
#--------------------------------------------------------------
# next step: look at the whole population data set to see what populations have the variant
#--------------------------------------------------------------
import pandas as pd

# 1) Load the genotype text file containing chromosome position, sample ID, genotype
genotype_file = '~/biometals/AnnotateVariants/var1_BetaThalassemiaInfo.txt' 
genotype_df = pd.read_csv(genotype_file, sep='\t', header=None)

# 2) Split the genotype file into separate columns
genotype_df[['position', 'sample', 'genotype']] = genotype_df[0].str.split(' ', expand=True)
# Drop the old column (0) after splitting
genotype_df = genotype_df.drop(columns=[0])

# 3) load samplepopdata
population_file = '~/biometals/samplepopdata.csv'
population_df = pd.read_csv(population_file, sep=',', header=None, names=['sample', 'pop'])

# 4) Merge the genotype data with samplepopdata
merged_df = pd.merge(genotype_df, population_df, on='sample', how='inner')

# 5) Filter for samples with the variation allele 
variation_samples = merged_df[merged_df['genotype'].str.contains('1\|1|1\|0|0\|1')]

# 6) Analyze the population distribution of these variation samples
population_counts = variation_samples.groupby('pop').size().reset_index(name='count')
print(population_counts)

   pop  count
0  PEL      1


In [ ]:
#--------------------------------------------------------------
# compare it to the total population data already determined from above (output pasted below)
#--------------------------------------------------------------

    pop  counts
0   ACB      96
1   ASW      61
2   BEB      86
3   CDX      93
4   CEU      99
5   CHB     103
6   CHS     105
7   CLM      94
8   ESN      99
9   FIN      99
10  GBR      91
11  GIH     103
12  GWD     113
13  IBS     107
14  ITU     102
15  JPT     104
16  KHV      99
17  LWK      99
18  MSL      85
19  MXL      64
20  PEL      85
21  PJL      96
22  PUR     104
23  STU     102
24  TSI     107
25  YRI     108

#--------------------------------------------------------------
#--------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------
# variant 2) ID==rs113506946 at POS==5285344
#--------------------------------------------------------------

In [1]:
#--------------------------------------------------------------
# Extracting variant positions for ID==rs113506946 at position 5285344
#--------------------------------------------------------------
!bcftools view -r chr11:5285000-5285500 -Ov -o ~/biometals/AnnotateVariants/chr11HBBbt_5.vcf ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz

In [1]:
!grep -v "^##" ~/biometals/AnnotateVariants/chr11HBBbt_5.vcf | head

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	HG00096	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103	HG00105	HG00106	HG00107	HG00108	HG00109	HG00110	HG00111	HG00112	HG00113	HG00114	HG00115	HG00116	HG00117	HG00118	HG00119	HG00120	HG00121	HG00122	HG00123	HG00125	HG00126	HG00127	HG00128	HG00129	HG00130	HG00131	HG00132	HG00133	HG00136	HG00137	HG00138	HG00139	HG00140	HG00141	HG00142	HG00143	HG00145	HG00146	HG00148	HG00149	HG00150	HG00151	HG00154	HG00155	HG00157	HG00158	HG00159	HG00160	HG00171	HG00173	HG00174	HG00176	HG00177	HG00178	HG00179	HG00180	HG00181	HG00182	HG00183	HG00185	HG00186	HG00187	HG00188	HG00189	HG00190	HG00231	HG00232	HG00233	HG00234	HG00235	HG00236	HG00237	HG00238	HG00239	HG00240	HG00242	HG00243	HG00244	HG00245	HG00246	HG00250	HG00251	HG00252	HG00253	HG00254	HG00255	HG00256	HG00257	HG00258	HG00259	HG00260	HG00261	HG00262	HG00263	HG00264	HG00265	HG00266	HG00267	HG00268	HG00269	HG00271	HG00272	HG00273	HG00274	HG00275	HG00276	HG00277	HG00278	HG00280	HG00281	HG00282	HG00284	HG

In [1]:
# Check if rs113506946 is in the file
!grep "rs113506946" ~/biometals/AnnotateVariants/chr11HBBbt_5.vcf # returns nothing, so it is not
# trying the position instead
!grep '5285344' ~/biometals/AnnotateVariants/chr11HBBbt_5.vcf

In [1]:
#--------------------------------------------------------------
# query variant sequence
#--------------------------------------------------------------
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'POS=="5285344" && GT="alt"' ~/biometals/AnnotateVariants/chr11HBBbt_5.vcf > ~/biometals/AnnotateVariants/var2_BetaThalassemiaInfo.txt

Error: cannot use arithmetic operators to compare strings and numbers


Conclusion: variant ID==rs113506946 at position 5285344 does not exist on the VCF file, given that it does not show up on multiple extracted files containing its location. 

#--------------------------------------------------------------
#--------------------------------------------------------------

In [ ]:
#--------------------------------------------------------------
# variant 3) ID==rs11549489 at POS==5288667 
#--------------------------------------------------------------

In [1]:
#--------------------------------------------------------------
# query variant sequence on chr11HBBbt.vcf
#--------------------------------------------------------------
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs11549489" && GT="alt"' ~/biometals/AnnotateVariants/chr11HBBbt.vcf > ~/biometals/AnnotateVariants/var3_BetaThalassemiaInfo.txt

In [1]:
# checking if the variant is present
!grep 'POS=="5288667"' ~/biometals/AnnotateVariants/chr11HBBbt.vcf
# looks like this variant is also not present

# Alpha Thalassemia

Located on HBA1 and HBA2 genes on Chr 16
Chr16:159,632-164,969 (HBA1 region, GRCh38)
Chr16:222,623-227,510 (HBA2 region, GRCh38)

In [1]:
# extract the HBA1 gene
!bcftools view -r chr16:159632-164969 -Ov -o ~/biometals/AnnotateVariants/1000Genomes.Chr16.HBA1.vcf ~/biol6150/Data/1000Genomes/phase3.chr16.GRCh38.GT.crossmap.vcf.gz

In [1]:
# query variant sequence
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs11041357" && GT="alt"' ~/biometals/AnnotateVariants/1000Genomes.Chr11.HMBS.vcf > ~/biometals/AnnotateVariants/var1_HBA1.txt

In [1]:
# extract the HBA2 gene
!bcftools view -r chr16:222623-227510 -Ov -o ~/biometals/AnnotateVariants/1000Genomes.Chr16.HBA2.vcf ~/biol6150/Data/1000Genomes/phase3.chr16.GRCh38.GT.crossmap.vcf.gz

In [1]:
# query variant sequence
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs63749907" && GT="alt"' ~/biometals/AnnotateVariants/1000Genomes.Chr11.HMBS.vcf > ~/biometals/AnnotateVariants/var1_HBA2.txt

neither of these worked

# Porphyria

# Acute Intermittent Porpyria (AIP) 

Acute Intermittent Porpyria comes from a variation located on the HMBS Gene on Chromosome 11. Thus, the chromosome 11 VCF file from above can be used, but the gene needs to be extracted first before analyzing the variant. 

In [1]:
# extract the HMBS gene
!bcftools view -r chr11:118050000-119092000 -Ov -o ~/biometals/AnnotateVariants/1000Genomes.Chr11.HMBS_2.vcf ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz

In [1]:
# query variant sequence
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs80338839" && GT="alt"' ~/biometals/AnnotateVariants/1000Genomes.Chr11.HMBS_2.vcf > ~/biometals/AnnotateVariants/var1_AIP.txt

ID: rs80338839
pos: 118548644

jk it didn't work

In [ ]:
# trying to just query the entire chromosome
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs80338839" && GT="alt"' ~/biol6150/Data/1000Genomes/phase3.chr11.GRCh38.GT.crossmap.vcf.gz > ~/biometals/AnnotateVariants/var1_AIP.txt

# Hereditary Hemochromatosis

Chromosome 6
2 variants: 
rs1800562: Chr6:26,072,300 
rs1799945: Chr6:26,071,919 

In [1]:
#--------------------------------------------------------------
# extract a smaller sequence that contains the gene
#--------------------------------------------------------------
!bcftools view -r chr6:26000000-26900000 -Ov -o ~/biometals/AnnotateVariants/1000Genomes.Chr6.vcf 
    ~/biol6150/Data/1000Genomes/phase3.chr6.GRCh38.GT.crossmap.vcf.gz

In [1]:
#--------------------------------------------------------------
# query for variant 1
#--------------------------------------------------------------
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs1800562" && GT="alt"' 
    ~/biometals/AnnotateVariants/1000Genomes.Chr6.vcf > ~/biometals/AnnotateVariants/chr6_var1.txt

In [1]:
#--------------------------------------------------------------
# next step: look at the whole population data set to see what populations have the variant
#--------------------------------------------------------------
import pandas as pd

# 1) Load the genotype text file containing chromosome position, sample ID, genotype
genotype_file = '~/biometals/AnnotateVariants/chr6_var1.txt' 
genotype_df = pd.read_csv(genotype_file, sep='\t', header=None)

# 2) Split the genotype file into separate columns
genotype_df[['position', 'sample', 'genotype']] = genotype_df[0].str.split(' ', expand=True)
# Drop the old column (0) after splitting
genotype_df = genotype_df.drop(columns=[0])

# 3) load samplepopdata
population_file = '~/biometals/samplepopdata.csv'
population_df = pd.read_csv(population_file, sep=',', header=None, names=['sample', 'pop'])

# 4) Merge the genotype data with samplepopdata
merged_df = pd.merge(genotype_df, population_df, on='sample', how='inner')

# 5) Filter for samples with the variation allele 
variation_samples = merged_df[merged_df['genotype'].str.contains('1\|1|1\|0|0\|1')]

# 6) Analyze the population distribution of these variation samples
population_counts = variation_samples.groupby('pop').size().reset_index(name='count')
print(population_counts)

    pop  count
0   ACB      1
1   ASW      2
2   CEU     10
3   CLM      3
4   FIN      9
5   GBR      8
6   GIH      1
7   IBS      9
8   MXL      1
9   PEL      2
10  PUR      8
11  STU      1
12  TSI      6


In [1]:
# query for variant 2
!bcftools query -f '[%CHROM:%POS %SAMPLE %GT\n]' -i 'ID=="rs1799945" && GT="alt"' ~/biometals/AnnotateVariants/1000Genomes.Chr6.vcf > ~/biometals/AnnotateVariants/chr6_var2.txt

In [1]:
#--------------------------------------------------------------
# next step: look at the whole population data set to see what populations have the variant
#--------------------------------------------------------------
import pandas as pd

# 1) Load the genotype text file containing chromosome position, sample ID, genotype
genotype_file = '~/biometals/AnnotateVariants/chr6_var2.txt' 
genotype_df = pd.read_csv(genotype_file, sep='\t', header=None)

# 2) Split the genotype file into separate columns
genotype_df[['position', 'sample', 'genotype']] = genotype_df[0].str.split(' ', expand=True)
# Drop the old column (0) after splitting
genotype_df = genotype_df.drop(columns=[0])

# 3) load samplepopdata
population_file = '~/biometals/samplepopdata.csv'
population_df = pd.read_csv(population_file, sep=',', header=None, names=['sample', 'pop'])

# 4) Merge the genotype data with samplepopdata
merged_df = pd.merge(genotype_df, population_df, on='sample', how='inner')

# 5) Filter for samples with the variation allele 
variation_samples = merged_df[merged_df['genotype'].str.contains('1\|1|1\|0|0\|1')]

# 6) Analyze the population distribution of these variation samples
population_counts = variation_samples.groupby('pop').size().reset_index(name='count')
print(population_counts)

    pop  count
0   ACB      4
1   ASW      6
2   BEB      5
3   CDX      4
4   CEU     25
5   CHB      6
6   CHS      5
7   CLM     25
8   ESN      1
9   FIN     20
10  GBR     31
11  GIH     17
12  GWD      1
13  IBS     45
14  ITU     20
15  JPT      4
16  KHV      7
17  LWK      2
18  MXL     14
19  PEL      9
20  PJL     11
21  PUR     23
22  STU     16
23  TSI     34
24  YRI      1
